In [4]:

import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
import numpy as np
import os
from PIL import Image
import warnings
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings("ignore")
 
plt.ion()



In [5]:
data_transforms = {
    'train': transforms.Compose([
        # 转换成tensor向量
        transforms.Grayscale(),
        transforms.Resize(84),
        transforms.ToTensor(),
        # 对图像进行归一化操作
        # [0.485, 0.456, 0.406]，RGB通道的均值与标准差
        transforms.Normalize(mean=[0.5], std=[0.5])
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(84),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ]),
}

def Load_Image_Information(path):
    # 图像存储路径
    image_Root_Dir= r'C:/Users/Administrator/Desktop/深度学习/BitmojiDataset/trainimages/'
    # 获取图像的路径
    iamge_Dir = os.path.join(image_Root_Dir, path)
    # 以RGB格式打开图像
    # Pytorch DataLoader就是使用PIL所读取的图像格式
    return Image.open(iamge_Dir).convert('L')

class my_Data_Set(nn.Module):
    def __init__(self, txt, transform=None, target_transform=None, loader=None):
        super(my_Data_Set, self).__init__()
        # 打开存储图像名与标签的txt文件
        #fp = open(txt, 'r')
        images = []
        labels = []
        data = pd.read_csv(txt)
        images=data["image_id"]
        labels=data["is_male"]
        labels[labels==-1]=0
        # 将图像名和图像标签对应存储起来
        #for line in fp:
            #line.strip('\n')
            #line.rstrip()
            #information = line.split()
            #images.append(information[0])
            #labels.append(int(information[1]))
        
        self.images = images
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
 
    # 重写这个函数用来进行图像数据的读取
    def __getitem__(self, item):
        # 获取图像名和标签
        imageName = self.images[item]
        label = self.labels[item]
        # 读入图像信息
        image = self.loader(imageName)
        # 处理图像数据
        if self.transform is not None:
            image = self.transform(image)
        return image, label
 
    # 重写这个函数，来看数据集中含有多少数据
    def __len__(self):
        return len(self.images)

    
def Load_Image_Information_val(path):
    # 图像存储路径
    image_Root_Dir= r'C:/Users/Administrator/Desktop/深度学习/BitmojiDataset/testimages/'
    # 获取图像的路径
    iamge_Dir = os.path.join(image_Root_Dir, path)
    # 以RGB格式打开图像
    # Pytorch DataLoader就是使用PIL所读取的图像格式
    return Image.open(iamge_Dir).convert('L')
    
    
class my_Data_Set_val(nn.Module):
    def __init__(self, txt, transform=None, target_transform=None, loader=None):
        super(my_Data_Set_val, self).__init__()
        # 打开存储图像名与标签的txt文件
        #fp = open(txt, 'r')
        images = []
        labels = []
        data = pd.read_csv(txt)
        images=data["image_id"]
        #labels=data["is_male"]
        #labels[labels==-1]=0
        # 将图像名和图像标签对应存储起来
        #for line in fp:
            #line.strip('\n')
            #line.rstrip()
            #information = line.split()
            #images.append(information[0])
            #labels.append(int(information[1]))
        
        self.images = images
        #self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
 
    # 重写这个函数用来进行图像数据的读取
    def __getitem__(self, item):
        # 获取图像名和标签
        imageName = self.images[item]
        #label = self.labels[item]
        # 读入图像信息
        image = self.loader(imageName)
        # 处理图像数据
        if self.transform is not None:
            image = self.transform(image)
        return image#, label
 
    # 重写这个函数，来看数据集中含有多少数据
    def __len__(self):
        return len(self.images)

In [6]:
train_dataset = my_Data_Set('train.csv', transform=data_transforms['train'], loader=Load_Image_Information)
test_dataset = my_Data_Set_val('val.csv', transform=data_transforms['val'], loader=Load_Image_Information_val)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
for data in train_loader:
    inputs,labels=data;
    #print(labels)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #输入图像为3*384*384
        self.conv1 = nn.Conv2d(1, 6, 5)
        #变为6*380*380
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 18 * 18, 800)
        self.fc2 = nn.Linear(800, 120)
        self.fc3 = nn.Linear(120, 10)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
 
        return x
 
net = Net()
 
#训练
cirterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
PATH="C:/Users/Administrator/Desktop/深度学习/"
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()                        # 优化器清零
        outputs = net(inputs)
        loss = cirterion(outputs, labels)
        loss.backward()
        optimizer.step()                         #优化
        running_loss += loss.item()
        if i % 50 == 49:
            print('[%d %5d] loss: %.3f' % (epoch + 1, (i + 1)*10, running_loss / 50))
            running_loss = 0.0
    P=os.path.join(PATH, str(epoch+1)+'.pth')
    torch.save(net.state_dict(), P)

 
print('finished training!')


[1   500] loss: 0.427
[1  1000] loss: 0.099
[1  1500] loss: 0.097
[1  2000] loss: 0.094
[1  2500] loss: 0.063
[1  3000] loss: 0.024
[2   500] loss: 0.034
[2  1000] loss: 0.044
[2  1500] loss: 0.051
[2  2000] loss: 0.007
[2  2500] loss: 0.008
[2  3000] loss: 0.041
[3   500] loss: 0.046
[3  1000] loss: 0.036
[3  1500] loss: 0.010
[3  2000] loss: 0.019
[3  2500] loss: 0.050
[3  3000] loss: 0.018
[4   500] loss: 0.034
[4  1000] loss: 0.009
[4  1500] loss: 0.027
[4  2000] loss: 0.016
[4  2500] loss: 0.037
[4  3000] loss: 0.008
[5   500] loss: 0.002
[5  1000] loss: 0.021
[5  1500] loss: 0.002
[5  2000] loss: 0.001
[5  2500] loss: 0.001
[5  3000] loss: 0.000
[6   500] loss: 0.000
[6  1000] loss: 0.010
[6  1500] loss: 0.001
[6  2000] loss: 0.000
[6  2500] loss: 0.000
[6  3000] loss: 0.000
[7   500] loss: 0.000
[7  1000] loss: 0.000
[7  1500] loss: 0.011
[7  2000] loss: 0.002
[7  2500] loss: 0.000
[7  3000] loss: 0.041
[8   500] loss: 0.131
[8  1000] loss: 0.025
[8  1500] loss: 0.008
[8  2000] 

In [8]:

#测试
correct = 0
total = 0
result=[]
result_2=[]
a=1
b=-1
with torch.no_grad():
    for data in test_loader:
        images = data
        outputs = net(Variable(images))
        _, predicted = torch.max(outputs.data, dim=1)
        for item in predicted.numpy():
            result.append(item)
            total=total+1
print(total)
data_csv = pd.read_csv(r"val.csv") # 读取刚才写入的文件
for item in result:
    if item==0:
        result_2.append(b)
    else:
        result_2.append(a)
np.savetxt('val.csv', result_2, delimiter=', ', fmt='%f')
#for item in result_2:
    #data_csv['is_male']=item
#data_csv.to_csv("val.csv", index=False, sep=',') # 将新增的列数据，增加到原始数据中

1084
